Creación de un estimador a partir de un modelo de Keras
===

* *30 min* | Última modificación: Abril 6, 2020

Basado en https://www.tensorflow.org/tutorials/estimator/keras_model_to_estimator

## Importación de librerías

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

tfds.disable_progress_bar()

print(tf.__version__)

#
# Establece el nivel de reporte en
# pantalla de TensorFlow
#
import logging

logger = tf.get_logger().setLevel(logging.ERROR)

2.4.1


## Carga y configuración del dataset

In [2]:
#
# Se crea una función para cargar el dataset
#
def input_fn():
    split = tfds.Split.TRAIN
    dataset = tfds.load("iris", split=split, as_supervised=True)
    dataset = dataset.map(lambda features, labels: ({"dense_input": features}, labels))
    dataset = dataset.batch(32).repeat()
    return dataset

In [3]:
for features_batch, labels_batch in input_fn().take(1):
    print(features_batch)
    print(labels_batch)

Dataset iris downloaded and prepared to /root/tensorflow_datasets/iris/2.0.0. Subsequent calls will reuse this data.
{'dense_input': <tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[5.1, 3.4, 1.5, 0.2],
       [7.7, 3. , 6.1, 2.3],
       [5.7, 2.8, 4.5, 1.3],
       [6.8, 3.2, 5.9, 2.3],
       [5.2, 3.4, 1.4, 0.2],
       [5.6, 2.9, 3.6, 1.3],
       [5.5, 2.6, 4.4, 1.2],
       [5.5, 2.4, 3.7, 1. ],
       [4.6, 3.4, 1.4, 0.3],
       [7.7, 2.8, 6.7, 2. ],
       [7. , 3.2, 4.7, 1.4],
       [4.6, 3.2, 1.4, 0.2],
       [6.5, 3. , 5.2, 2. ],
       [5.5, 4.2, 1.4, 0.2],
       [5.4, 3.9, 1.3, 0.4],
       [5. , 3.5, 1.3, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [4.8, 3. , 1.4, 0.1],
       [6.5, 3. , 5.8, 2.2],
       [7.6, 3. , 6.6, 2.1],
       [6.7, 3.3, 5.7, 2.1],
       [7.9, 3.8, 6.4, 2. ],
       [6.7, 3. , 5.2, 2.3],
       [5.8, 4. , 1.2, 0.2],
       [6.3, 2.5, 5. , 1.9],
       [5. , 3. , 1.6, 0.2],
       [6.9, 3.1, 5.1, 2.3],
       [6.1, 3. , 4.6, 1.4],
      

## Especificación del modelo

In [4]:
keras_model = keras.models.Sequential(
    [
        keras.layers.Dense(16, activation="relu", input_shape=(4,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(3, activation="sigmoid"),
    ]
)

keras_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
)

keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


## Especificación del estimador

In [5]:
import tempfile

model_dir = tempfile.mkdtemp()

keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=keras_model, model_dir=model_dir
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


## Entrenamiento

In [6]:
keras_estimator.train(input_fn=input_fn, steps=500)

eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)

print("Eval result: {}".format(eval_result))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Eval result: {'loss': 0.46195364, 'global_step': 500}
